In [14]:
from collections import defaultdict
import datetime
import json
import logging
import sys

import numpy as np
import pandas as pd

In [3]:
sys.path.append('Toggl.py')

In [4]:
from toggl import Toggl
import settings

In [5]:
# timeframe for report
end_date = datetime.datetime.now()
start_date = end_date - settings.timeframe

In [20]:
# create report
report_builder = Toggl(settings.api_token)
workspaces = report_builder.get_workspaces()

reports = []

In [21]:
for ws_name, ws_id in workspaces:
    if ws_name in settings.workspace2meta.keys():
        metaproject = settings.workspace2meta[ws_name]

        for record in report_builder.detailed_report(ws_id, start_date, end_date):
            # record duration is in milliseconds
            # divide by 3600000 to convert to hours
            reports.append({
                'user': record['user'],
                'team': ws_name,
                'project': metaproject,
                'subproject': record['project'],
                # example of record['start']: 2015-05-29T16:07:20+03:00
                'start': record['start'][:19],
                'duration': round(float(record['dur']) / 3600000, 2)
            })

In [25]:
df = pd.DataFrame(reports)
df['start'] = pd.to_datetime(df['start'])

In [26]:
df.head()

,duration,project,start,subproject,team,user
0,0.09,YORSO,2017-01-25 09:22:51,Прочее,YORSO,Stepan Nikiforov
1,0.82,YORSO,2017-01-25 09:30:50,WEB,YORSO,Igor Sakhankov
2,0.25,YORSO,2017-01-25 09:40:15,Прочее,YORSO,Stepan Nikiforov
3,0.21,YORSO,2017-01-25 10:00:03,Прочее,YORSO,Stepan Nikiforov
4,0.13,YORSO,2017-01-25 10:12:43,Прочее,YORSO,Stepan Nikiforov


In [27]:
project_daily = df.set_index('start').groupby('project').resample('1D').sum()

In [36]:
project_daily.head()

duration
project start               
YORSO   2017-01-25     20.77
        2017-01-26     17.93
        2017-01-27     13.27
        2017-01-28       NaN
        2017-01-29      2.30

In [40]:
project_daily.loc[u"Минимакс"]['duration']

start
2017-01-25    12.11
2017-01-26    17.65
2017-01-27    13.02
2017-01-28     5.67
2017-01-29     2.62
2017-01-30    17.04
2017-01-31    17.47
2017-02-01    13.72
Freq: D, Name: duration, dtype: float64

In [50]:
json.dumps(project_daily.loc[u"Минимакс"]['duration'].values.tolist())

'[12.110000000000001, 17.65, 13.019999999999998, 5.67, 2.62, 17.040000000000006, 17.469999999999995, 13.719999999999999]'

In [51]:
user_daily = df.set_index('start').groupby('user').resample('1D').sum()

In [52]:
user_daily.head()

duration
user               start               
Aleksandr Shepelev 2017-01-25      3.06
                   2017-01-26      4.95
                   2017-01-27      3.45
                   2017-01-28      2.86
                   2017-01-29      2.62

In [58]:
dti = user_daily.index.get_level_values(1).unique()

In [64]:
dti.map(lambda x: x.date())

array([datetime.date(2017, 1, 25), datetime.date(2017, 1, 26),
       datetime.date(2017, 1, 27), datetime.date(2017, 1, 28),
       datetime.date(2017, 1, 29), datetime.date(2017, 1, 30),
       datetime.date(2017, 1, 31), datetime.date(2017, 2, 1)], dtype=object)

In [68]:
json.dumps(dti.map(lambda x: str(x.date())).tolist())

'["2017-01-25", "2017-01-26", "2017-01-27", "2017-01-28", "2017-01-29", "2017-01-30", "2017-01-31", "2017-02-01"]'

In [66]:
tmp = dti[0].date()

In [67]:
str(tmp)

'2017-01-25'